# Neural Networks: Simple

## Chapter 7

### Dot Product

In [1]:
from typing import List
from math import exp

def dot_product(xs: List[Float], ys: List[Float]) -> float:
    """
    Dot product of two lists.
    """
    return sum(x * y for x, y in zip(xs, ys))

### Sigmoid Activation Function

In [2]:
def sigmoid(x: float) -> float:
    """
    Sigmoid activation function.
    """
    return 1 / (1 + exp(-x))

def derivative_sigmoid(x: float) -> float:
    """
    Derivative of sigmoid activation function.
    """
    return sigmoid(x) * (1 - sigmoid(x))

## Neuron

Must have 

* weights
* delta
* learning_rate
* cache of last output
* activation_function
* derivative of activation_function

In [3]:
from typing import Callable


class Neuron:
    """
    Basic unit for a neural network. Must have weights, activation function, derivative of activation function,
    learning rate, cache of last output, and delta.
    """

    def __init__(
        self,
        weights: List[float],
        learning_rate: float,
        activation_function: Callable[[float], float],
        derivative_activation_function: Callable[[float], float],
    ) -> None:
        self.weights: List[float] = weights
        self.activation_function: Callable[[float], float] = activation_function
        self.derivative_activation_function: Callable[
            [float], float
        ] = derivate_activation_function
        self.learning_rate: float = learning_rate
        self.output_cache: float = 0.0
        self.delta: float = 0.0

    def output(self, inputs: List[float]) -> float:
        """
        Feed forward pass of neuron.
        """
        self.output_cache = dot_product(inputs, self.weights)
        return self.activation_function(self.output_cache)

## Layer

A layer must have:

* neurons
* output cache (after the activation function is applied to neuron's output)
* previous layer

In [ ]:
from __future__ import annotations
from typing import Optional
from random import random


class Layer:
    """
    Base class for a neural network layer. Must know the previous layer, the neurons, and an output cache (after activation function).
    """

    def __init__(
        self,
        previous_layer: Optional[Layer],
        num_neurons: int,
        learning_rate: float,
        activation_function: Callable[[float], float],
        derivative_activation_function: Callable[[float], float],
    ) -> None:
        self.previous_layer: Optional[Layer] = previous_layer
        self.neurons: List[Neuron] = []

        # Add neurons
        for i in range(num_neurons):
            if previous_layer is None:
                random_weights: List[float] = []
            else:
                # Each neuron is connected to every neuron in previous layer
                random_weights = [random() for _ in range(len(previous_layer.neurons))]
            neuron: Neuron = Neuron(
                random_weights,
                learning_rate,
                activation_function,
                derivative_activation_function,
            )
            self.neurons.append(neuron)
        # Initialize empty output cache
        self.output_cache: List[float] = [0.0 for _ in range(num_neurons)]

    def outputs(self, inputs: List[float]) -> List[float]:
        """
        Calculate outputs of all neurons in layer.
        """
        if self.previous_layer is None:  # This is an input layer
            self.output_cache = inputs
        else:
            self.output_cache = [n.output(inputs) for n in self.neurons]
        return self.output_cache

    def calculate_deltas_for_output_layer(self, expected: List[float]) -> None:
        """
        Calculate deltas for output layer. Delta = f'(output_cache) * error
        
        f'(output_cache) is derivative of activation with respect to output cache
        error is expected - actual
        """
        for n in range(len(self.neurons)):
            # Call the derivative of the activation function on the neuron's output and
            # multiply by the expected value
            self.neurons[n].delta = self.neurons[n].derivative_activation_function(
                self.neurons[n].output_cache
            ) * (expected[n] - self.output_cache[n])
            
    def calculate_deltas_for_hidden_layer(self, next_layer: Layer) -> None:
        """
        Calculate deltas for hiddne layer. Delta = f'(output_cache) * (next_weights * next_deltas)
        """
        for index, neuron in enumerate(self.neurons):
            next_weights: List[float] = [n.weights[index] for n in next_layer.neurons]
            next_deltas: List[float] = [n.delta for n in next_layer.neurons]
            sum_weights_and_deltas: float = dot_product(next_weights, next_deltas)
            neuron.delta = self.derivative_activation_function(neuron.output_cache) * sum_weights_and_deltas